This notebook will use evaluation code to make some plots of MARS's performance on test data.

In [1]:
from evaluation import eval
import os
import json

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
summary_dir = 'D:\\MARS_eval_test\\'
config = 'D:\\MARS_data\\tfrecords\\tfrecords_pose_top\\config_test.yaml'
cfg = eval.parse_config_file(config)
parts = cfg['PARTS']['NAMES']
print(parts)

with open(os.path.join(summary_dir,'MARS_results_CoCo.json')) as jsonfile:
    data = json.load(jsonfile)

['nose tip', 'right ear', 'left ear', 'neck', 'right side body', 'left side body', 'tail base']


In [5]:
eval.eval_coco(gt_keypoints=data['gt_keypoints'], pred_keypoints=data['pred_keypoints'], parts=parts)

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*


TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [ ]:
tfrecords = ['D:\\MARS_data\\tfrecords\\tfrecords_pose_top\\test_dataset-00000-of-00001']
checkpoint_path = 'X:\\cristina_code_original\\results_keypoints\\top_fixed'
summary_dir = 'D:\\MARS_eval_test\\'
config = 'D:\\MARS_data\\tfrecords\\tfrecords_pose_top\\config_test.yaml'

# these are optional inputs to eval- set to True if you would like it to generate heatmaps of all keypoints:
max_iterations=0          # only run on the first X images (set to 0 to run on all data)- useful for debugging
show_heatmaps=False       # heatmaps from final layer only
show_layer_heatmaps=False # show heatmaps from all layers of the stacked hourglass



eval.eval(tfrecords, checkpoint_path, summary_dir, config,
          max_iterations=2, show_heatmaps=show_heatmaps, show_layer_heatmaps=show_layer_heatmaps)

X:\cristina_code_original\results_keypoints\top_fixed
is now:X:\cristina_code_original\results_keypoints\top_fixed\model.ckpt-700000
INFO:tensorflow:Restoring parameters from X:\cristina_code_original\results_keypoints\top_fixed\model.ckpt-700000
